# Unsupervised Text Analysis

#### Introduction
In this exercise, we will explore and analyze text data in an unsupervised manner. Our corpus will be Amazon reviews of a specific product, including the free-form text review and the star rating.

What can we learn from the data that would add value to the business?
* What are people saying about the product?
* What do people love about the product?
* What do people hate about the product?


## Contents
* 1) Library and Data Imports
* 2) Quick Exploratory Analysis
* 3) Text Cleaning
* 4) Text EDA
* 5) Text Analysis
* * a) Topic Modelling
* * b) Phrase Frequency Analysis
* Optional) Contextual (metadata) EDA

# 1) Library and Data Imports

In addition to the standard Python libraries such as Pandas, Numpy, and Matplotlib, we will import will use the text analytics library NLTK (Natural Language Toolkit) as well as some text processing functions from scikit-learn for our text preprocessing and EDA.

NLTK is a text analytics library, providing an interface to corpora and lexical resources as well as multiple text preprocessing modules for tokenization, stemming, lemmatization, parsing, chunking, POS tagging, removing stop words, etc. It is purely for symbolic and statistical NLP.

*What is a corpus? In NLP, the dataset of texts is called the "corpus", and each text within the corpus is referred to as a "document".*

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import re
import random
from operator import itemgetter

print("Importing NLTK...")
from nltk import FreqDist
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
sw = stopwords.words("english")

print("Importing Modules from Sklearn...")
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

In [ ]:
df = pd.read_csv('amazon_home_v1_00_reviews_B00009R66F.csv')
text_col = 'review_body'

Bag-of-Words (BOW) Methods:

Most statistical NLP approaches are based on a "bag-of-words" approach, in which a document is considered as an unordered set of words. Each word is considered mainly as a symbol and is devoid of semantic meaning. That is, the computer doesn't consider "good" and "great" to be any more similar to each other than "good" and "goat". Moreover, a word with capital letters or punctuation is a completely different symbol than the same word without capital letters or punctuation. "Good!", "Good", and "good" will be treated as three different words.

For this reason, BOW approaches require certain standard text cleaning steps, such as lowercasing and punctuation removal. Additionally, since no word is considered as inherently more "meaningful" than any other at the start of the analysis, it is important to remove very common words that don't add meaning. These words, such as "is", "the", etc. are called "stop words".

# 2) Quick Exploratory Analysis
The purpose of this exploratory step is to determine what text cleaning steps we need to take in addition to the usual lowercasing, punctuation removal, and stop word removal. Does our text contain urls, @mentions, or #hashtags, and if so, are they potentially important "words", or should we remove them? Does the text contain html tags? Email addresses? What other "noise" exists in our text?

In [ ]:
def show_random_texts(df, text_col, n_texts=5):
    rands = [random.randint(0,len(df)) for i in range(n_texts)]
    texts = df.iloc[rands][text_col].to_list()
    for t in texts:
        print(t)
        print()

In [ ]:
# Call `show_random_texts()` on your dataframe
### YOUR CODE HERE (1 line)

#### We might notice a few "messy" things about our text data, such as:
* Extra white spaces
* Double backslash
* HTML break tag

#### Let's check for HTML tags, hashtags, etc.:

In [ ]:
tags, mentions, hashtags = 0, 0, 0
for t in df[text_col].to_list():
    if (tags > 1) and (hashtags > 1): # We just want to print a sampling of the texts, so we'll stop once we have found at least one HTML tag and one hashtag
        break
    # Check for HTML tags (texts containing "<")
    elif "<" in t:
        print(t)
        print()
        tags += 1
    
    # Check for hashtags (texts containing "#")
    ### YOUR CODE HERE (about 4 lines)
    
    # Check for mentions (texts containing "@")
    ### YOUR CODE HERE (about 4 lines)

*We see some break tags and URLs, as well as the code `&#34;`, which is HTML encoding for the double quotation mark.*

### Repeated words are fairly common typos within survey and review data. Let's check for them:

In [ ]:
def find_repeated_words(text):
    txt = text.split()
    count=0
    for i in range(1, len(txt)):
        if txt[i] == txt[i-1]:
            print(txt[i])
            print(text)
            print()
            count += 1
            break
    return count
   
num_texts_with_repeated_words = 0

# Check for repeated words in the first 50 rows of our dataframe
for t in df[text_col].iloc[:50].to_list():
    c = find_repeated_words(t)
    # Add result to num_texts_with_repeated_words
    ### YOUR CODE HERE (1 line)
    
print("Total texts containing repeated words in first 50 texts:", num_texts_with_repeated_words)

You might see that some words are repeated. These seem to mainly be typos, so we can remove them.

#### Are there any duplicate reviews?
We will define duplicate reviews as two or more identical reviews having 4 or more words. It is likely that multiple different might write the same generic 1 to 3-word review (e.g. "Great" or "Easy to use"), so we shouldn't consider these as duplicates

In [ ]:
%%time
reviews_vc = df['review_body'].value_counts().reset_index().rename(columns={"review_body":"count", "index":"review_body"})
reviews_vc['word_count'] = reviews_vc['review_body'].map(lambda x: len(x.split()))
# Define repeat_reviews as reviews that have a "count" greater than 1 and a "word_count" of at least 4
### YOUR CODE HERE (1 line)

print("Number of duplicate reviews:", len(repeat_reviews))
if (len(repeat_reviews) > 0) & (len(repeat_reviews) < 10):
    print(repeat_reviews)

# 3) Text Cleaning
The first step to any bag-of-words (BOW) text analysis approach is to clean your data. Since the computer will treat "Python", "python", and "python." as three unique words, we need to lowercase our text and remove punctuation. We also found that our text contained URLs and HTML tags. We use regexes to remove these before removing punctuation.

Additionally, we need to remove any words that don't add meaning, that is, that don't help tell us what the text is about. These words, such as "a", "the", "that", are called stop words. Your text might also contain domain-specific stop words, words that appear in nearly every document and that are not going to help distinguish between documents or tell you what the documents are about. You would want to add these to the list of generic stopwords.

Additionally, we want different forms of the same word (e.g. "jump", "jumping", "jumped") to be considered as the same word. To achieve this, we use stemming or lemmatization. Stemming cuts off the last digits of a word based on certain rules (e.g. "hoped" --> "hop"), whereas lemmatization converts the word to its root form (e.g. "hoped" --> "hope", "won" --> "win"). Lemmatization is usually preferable, since it always outputs real words.

We will define a function to clean our texts using these techniques as well as those cleaning steps we defined for our particular dataset based on the EDA above.

Cleaning Steps:
* Normalize words and remove noise (lowercase, remove punctuation, remove URLs, HTML tags and entities, etc.)
* Lemmatize
* Remove Stopwords *(we do this after lemmatization because the POS tagger relies on stopwords for accurate tagging, and the lemmatizer requires POS tags for correct lemmatization)*
* Remove repeated words

In [ ]:
def clean_amazon_text(text):
    """Cleans text but does not remove stopwords. Follow this cleaning with POS tagging and lemmatization.
    Remove stopwords during or after the lemmatization step."""
    
    # Lowercase the text
    ### YOUR CODE HERE (1 line)
    
    
    # Remove URLs
    url_pattern = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    urls = [u[0] for u in re.findall(url_pattern, text)]
    text = re.sub(url_pattern, ' ', text)
    
    # Remove html tags
    text = re.sub('<[A-Za-z0-9 /]+>', ' ', text)
    text = re.sub('<[A-Za-z0-9\s=/"-_]+>', ' ', text)
    
    # Remove "&#34;" and any other HTML entities
    text = text.replace("&#\d*;", " ")
    
    # Remove non-alphanumeric characters
    text = re.sub("[\W+']", ' ', text)
    text = re.sub("_", ' ', text)
    
    # Replace multiple whitespaces with a single whitespace
    text = re.sub("\s+", ' ', text).strip()
    
    return text

Lemmatization can be done using NLTK. Lemmatization utilizes POS tags (part of speech tags, telling whether the word is a noun, adjective, verb, etc.) to follow the appropriate rules for lemmatizing each. The default NLTK POS tags, however, do not work for the NLTK WordnetLemmatizer, so we first define a function to convert the POS tags to the proper format.

In [ ]:
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.ADV

def lemmatize_text(text_str, sw=sw):
    from nltk.stem import WordNetLemmatizer
    lm = WordNetLemmatizer()

    # Tokenize the text (split them into lists of words)
    ### YOUR CODE HERE (1 line)

    text_tagged = [t for t in nltk.pos_tag(text_tok) if t[0] not in sw]
    text_lem = ' '.join(lm.lemmatize(w[0], pos=get_wordnet_pos(w[1])) for w in text_tagged)
    return text_lem

In [ ]:
def remove_repeated_words(text):
    """Removes consecutive repeated occurrences of words within a string, leaving only the first occurrence.
    
    text: (string)
    

Example:
    ---------
    >>> text = remove_repeated_words("I really really really like data science.")
    >>> print(text)
    I really like data science.
    
    """
    
    to_remove = []
    txt = text.split()
    for i in range(1, len(txt)):
        # Check whether a word is the same as the word before it, and if it is, append i to to_remove
        ### YOUR CODE HERE (2 lines)
        if txt[i] == txt[i-1]:
            to_remove.append(i)


    # Remove words by index (so remove in reverse order to keep the indices correct)
    to_remove.reverse()
    for i in to_remove:
        txt.pop(i)
    return ' '.join(t for t in txt)

In [ ]:
# Test our cleaning functions
text = "I really <br> liked <br /> my my new hoover cleaner until it broke. It's only a month after I bought it! www.hoover.com!"

text = clean_amazon_text(text)
print("Cleaned text:", text)
text = lemmatize_text(text)
print("Lemmatized and removed stopwords:", text)
text = remove_repeated_words(text)
print("Cleaned text with repeated words removed:", text)

One other cleaning step we might want to do is to remove the product name and id from the reviews

In [ ]:
if df['product_title'].nunique() == 1:
    title = clean_amazon_text(df['product_title'].unique()[0])
else:
    title = ''
    
if df['product_id'].nunique() == 1:
    prod_id = df['product_id'].unique()[0].lower()
else:
    prod_id = ''
    
title, prod_id

In [ ]:
def remove_product_mentions(text, title, prod_id):
    """
    Optional text cleaning step to remove product ids and product name mentions from the reviews. Removes words only if the word plus the following word or the previous word plus the word are in the title.
    """

    # Filter Noise - Remove product id and words in product name
    text = re.sub(prod_id.lower(), ' ', text)
    to_remove = [i for i in range(len(text.split())-1) if ' '.join([text.split()[i],text.split()[i+1]]) in title] # remove word if word plus following word are in title
    to_remove += [i for i in range(1,len(text.split())) if ' '.join([text.split()[i-1],text.split()[i]]) in title] # remove word if previous word plus word are in title
    text = ' '.join(text.split()[i] for i in range(len(text.split())) if (i not in to_remove) & (text.split()[i] != prod_id))

    return text

# test the function
remove_product_mentions('I really like my new hoover steamvac carpet cleaner clean surge f5914900 its a great b00009r66f purchase f5914 900',
                        title='hoover steamvac carpet cleaner with clean surge f5914900', prod_id='b00009r66f')

##### Apply Cleaning Functions to Dataset

In [ ]:
%%time 
# This cell takes a few minutes to run

### Finish four lines of code in this cell

import time
# Clean texts in dataframe
t1 = time.time()
df[text_col+"_cleaned"] = ### YOUR CODE HERE
t2 = time.time()
print("Finished basic cleaning...", t2-t1)

# Lemmatize text
df[text_col+"_cleaned"] = df[text_col+"_cleaned"].map(###YOUR CODE HERE
t3 = time.time()
print("Lemmatized text...", t3-t2)

# Remove product titles and product IDs
df[text_col+"_cleaned"] = ###YOUR CODE HERE
t4 = time.time()
print("Removed product titles and IDs...", t4-t3)

# Remove repeated words
df[text_col+"_cleaned"] = ###YOUR CODE HERE
t5 = time.time()
print("Removed repeated words", t5-t4)

df[text_col+"_cleaned"]

# 4) Text EDA
* Review lengths
* Review length distribution by rating
* Review length distribution over time
* Most frequent words

### Review Lengths

Let's calculate the word count of each text, and then plot a histogram of the word counts. We will find that there are a few outliers (texts that are much longer than the rest), so we will plot the actual histogram as well as a histogram of the bottom 95% of texts

In [ ]:
# Make a column containing each text's word count
df[text_col+'_wc'] = df[text_col+"_cleaned"].str.split().str.len()

In [ ]:
def plot_wordcount(counts, percentile_max=0.95, text_raw_cleaned='Cleaned'):
    """
    Distribution of word counts of all raw texts and of word counts up to the percentile_max percentile.

    percentile_max: (float between 0 & 1, default=0.95)
    """
    n_words = int(counts.quantile(percentile_max))

    fig, ax = plt.subplots(1, 2, figsize=(15,6))
    ax[0].hist(counts)
    ax[0].set(xlabel=f'{text_col.title()} Word Count', ylabel= "Frequency", title=f'Word Count of All {text_raw_cleaned} {text_col.title()}')
    ax[1].hist(counts[counts<n_words])
    ax[1].set(xlabel=f'{text_col.title()} Word Count', ylabel= "Frequency", title=f'Word Count of {text_raw_cleaned} {text_col.title()} Shorter than {n_words} Words')
    plt.show()

In [ ]:
plot_wordcount(df[text_col+'_wc'])

#### Do our customers tend to write longer reviews when they are happy or unhappy with the product?

In [ ]:
def boxplots(df, text_col, col=None):
    """Boxplots of word count, split by values in col

    col: column name or None (default None)

    """
    if (not col) | (not col in df.columns):
        col = input(f"Which column do you want to use for x: {df.columns}")

    if not text_col+'_wc' in df.columns:
        print(f"Using cleaned {text_col}")
        if text_col+"_cleaned" in df.columns:
            df[text_col+'_wc'] = df[text_col+"_cleaned"].str.split().str.len() # or df[text_col].map(lambda x: len(x.split()))
        else: 
            print(f"Using raw {text_col}")
            df[text_col+'_wc'] = df[text_col].str.split().str.len()
    
    fig, ax = plt.subplots(1, 2, figsize=(15,6))
    sns.boxplot(x=df[col], y=df[text_col+'_wc'], ax=ax[0])
    ax[0].set(xlabel=col.title(), ylabel=f'{text_col.title()} Word Count', title=f'Actual Word Count of {text_col.title()} by {col.title()}')

    sns.boxplot(x=df[col], y=np.log(df[text_col+'_wc']), ax=ax[1])
    ax[1].set(xlabel=col.title(), ylabel=f'Log of {text_col.title()} Word Count', title=f'Log Word Count of {text_col.title()} by {col.title()}')

    plt.show()

In [ ]:
boxplots(df, text_col, 'star_rating')

#### Has the review length changed over time?

In [ ]:
df['review_date'] = pd.to_datetime(df['review_date'])
df['year'] = df['review_date'].map(lambda x: x.year)

boxplots(df, text_col, 'year')

You might notice that happy customers tend to write shorter reviews. Even more noticeable is that in the last 3 years, the average review length decreased drastically. While there could be many factors at play, it is seeing whether the average rating has been increasing over these three years.

#### Most Frequent Words

In [ ]:
# Plot top 10 most frequent words
def plot_most_frequent_words(texts, title="", col=None, value=None):
    freqs = FreqDist(" ".join(t for t in texts).split())
    word_freq = pd.Series(freqs).sort_values(ascending=True)
    plt.figure(figsize=(6,4))
    plt.barh(word_freq[-10:].index, word_freq[-10:].values)
    plt.xlabel('Word Frequency')
    if (col is not None) & (value is not None):
        plt.title(f"Word Frequency of Top {len(word_freq[-10:])} Words: {col}={value} " + title)
    else:
        plt.title(f"Word Frequency of Top {len(word_freq[-10:])} Words " + title)
    plt.show()
    
plot_most_frequent_words(df['review_body_cleaned'], "\n(All Texts)")

In [ ]:
# Plot top words by rating

# Get word frequencies by rating
texts_by_rating = {}
for r in df['star_rating'].unique():
    texts_by_rating[r] = df[df['star_rating']==r]['review_body_cleaned'] 

for r in sorted(texts_by_rating.keys()):
    plot_most_frequent_words(texts_by_rating[r], f"\nRating={str(r)}")

# 5) Text Analysis
Now we finally get into the heart of the matter, analyzing the text to answer questions such as:
* What are people saying about the product? 
* What do they like? 
* What do they dislike?

One drawback of BOW approaches is that words are mere symbols. The computer has no way of treating "good" and "great" as more similar to each other than "good" and "apple". While there are more advanced ways to retain semantic understanding in NLP, such as word2vec and deep learning approaches, we will see how far we can get with the BOW approach.

We will analyze our text using two approaches:
* a) Topic Modelling
* b) Phrase Frequency Analysis

### a) Topic Modelling
What is Topic Modelling? When we use the word "topic", we generally mean the subject that the conversation or text is about. Topic modelling is an unsupervised approach for discovering what documents are about, using the words that occur in them. In NLP, "topic" has a specific meaning, namely, a group of words that go together. These sets of words are usually determined by considering the co-occurrences of words.

There are multiple topic modelling algorithms, including LDA, NMF, CorEx, and Gibbs Sampling Dirichlet Mixture Model (GSDMM). Probably the most well-known is LDA. 

LDA is a probabilistic model that models how the comments were generated. It considers documents as combinations of topics in a certain proportion, and topics as groups of words, and it outputs a probability distribution of topics for each document and of words for each topic.

##### Topic Modeling using Gensim

Gensim is a Python library for topic modelling, text vectorization, and similarity retrieval.

To create a topic model in Gensim, after preprocessing the text using standard BOW approaches, we need to represent each document as a list of words. Then we can convert this into a Gensim dictionary, which gives each unique word a numberic id.

In [ ]:
%%time
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

##### Prepare Data for Gensim

In [ ]:
# Tokenize the texts in df['review_body_cleaned'] (i.e. split them into lists of words). 
# Define variable called texts_split, which is a list of the tokenized texts

texts_split = df['review_body_cleaned'].map(lambda x: x.split()).to_list()

One way to combat the deficiencies of the BOW approach is by combining words into pairs or phrases. In NLP, groups of consecutive words are called n-grams, where "n" specifies the number of words. "extremely fast" is a bigram, "cleans extremely fast" is a trigram or 3-gram, and so on.

`gensim.models.phrases.Phrases` "detects phrases (bigrams) based on collocation counts." That is, based on the number of times two words occur consecutively. By adjusting the min_count (which "ignores all words and bigrams with total collected count lower than this value") and threshold (higher threshold yields a smaller number of phrases), you can adjust the number and quality of your bigrams.

In [ ]:
# Join words into ngrams using Gensim
ngrams = gensim.models.phrases.Phrases(texts_split, min_count=30, threshold=10) ### Can tweak these parameters to get better ngrams

ngram_lst = []
ngram_texts = [ngrams[texts_split[i]] for i in range(len(texts_split))]

In [ ]:
# Print your ngrams so you can see if they make sense. Change min_count and threshold in the cell above to see if you get a better set of ngrams
for b in ngram_texts:
    for w in b:
        if "_" in w:
            ngram_lst.append(w)
            
ngrams = set(ngram_lst)
print(len(ngrams))
print(ngrams)

Now we can create our Gensim dictionary. The dictionary has certain methods for easily preparing data for topic modelling. We use `filter_extremes()` to filter out all words that occur less than a specified number of times, and we create a corpus numeric tuples representing the words and their frequencies in each text.

In [ ]:
dictionary_ngram = corpora.Dictionary(ngram_texts)
print(dictionary_ngram)
dictionary_ngram.filter_extremes(no_below=5) # Excluding any words that occur in less than 5 documents
print(dictionary_ngram)

corpus = [dictionary_ngram.doc2bow(text) for text in ngram_texts]

In [ ]:
corpus[0]

Now we can fit our LDA topic model.

Note that LDA requires us to choose and input the desired number of topics as a parameter. As you don't normally know the number of topics that exist in your data, in the real world, you will want to try multiple values for num_topics and choose the model that performs the best.

In [ ]:
%%time 
# This cell takes a couple minutes to run.
random_state = 12
num_passes = 5
NUM_TOPICS = 20

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary_ngram, 
                                           passes=num_passes, random_state=random_state, per_word_topics=True)
topics = ldamodel.print_topics(num_words=10) # You can have it print more or fewer words if you want

Let's take a look at our topics. As mentioned earlier, topics in NLP are groups of words. We will print the 10 most important words of each topic as a list.

Feel free to try another value for NUM_TOPICS to see how your topics change and whether you get better topics.

In [ ]:
def get_topic_info(topic):
    if type(topic) == tuple:
        topic = topic[1]
    topic_lst = topic.split(" + ")
    words = [t.split("*")[1].strip('"') for t in topic_lst]
    weights = [t.split("*")[0] for t in topic_lst]
    return words, weights

In [ ]:
topic_words = []
topic_weights = []
for t in topics:
    words, weights = get_topic_info(t)
    topic_words.append(words)
    topic_weights.append(weights)

for i, t in enumerate(topic_words):
    print(f"Topic {i}", '\t', t)

#### Interpreting the topics
Briefly look over the words in each topic, and see if you can come up with titles for them

* Topic 0: ### YOUR TITLES HERE
* Topic 1: 
* Topic 2: 
* Topic 3: 
* Topic 4: 
* Topic 5: 
* Topic 6: 
* Topic 7: 
* Topic 8: 
* Topic 9: 
* Topic 10: 
* Topic 11: 
* Topic 12: 
* Topic 13: 
* Topic 14: 
* Topic 15:
* Topic 16: 
* Topic 17: 
* Topic 18:
* Topic 19:

### Evaluate Topic Models

Since topic models are an unsupervised learning approach, we don't have labels and cannot evaluate our model using supervised learning metrics such as accuracy, precision, and recall. Instead, we evaluate an LDA model using `coherence`, which is a measure of how well-separated and cohesive the learned topics are. The higher the coherence score, the more the words in the topic should "make sense" together.

In [ ]:
%%time
# This cell takes a few minutes to run
coherence_model_lda = CoherenceModel(model=ldamodel, texts=ngram_texts, dictionary=dictionary_ngram, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print("Coherence Score:", round(coherence_lda,3))

In [ ]:
coherence_per_topic = coherence_model_lda.get_coherence_per_topic()
sort_per_topic=[]

for i in range(len(coherence_per_topic)):
    sort_per_topic.append((i, coherence_per_topic[i]))
    
sort_per_topic = sorted(sort_per_topic, key=itemgetter(1), reverse=True)

# bar plot

X = [w[0] for w in sort_per_topic]
Y = [w[1] for w in sort_per_topic]
plt.bar(x=X, height=Y, width=0.8)
plt.xlabel("Topic Number")
plt.ylabel("Topic Coherence Score")
plt.title("Topic Coherences")
plt.show()

An additional tool for evaluating your topic model is pyLDAvis. pyLDAvis visualizes your topics as circles and shows the top words in each topic. The more separated the circles are, the better the model is. For the sake of time, we won't go over it here, but I encourage you to check it out if you want to learn more about topic modelling. 
* https://pyldavis.readthedocs.io/en/latest/ 
* https://github.com/bmabey/pyLDAvis

### Topic Assignments

Another way we can get a feel for how well our topics are representing the data is by looking at some of the documents that primarily contain a certain topic. While the documents are assumed to contain multiple topics, we can "assign" each document to the topic which has the highest probability.

In [ ]:
doc_topics = [np.argmax([d[1] for d in ldamodel.get_document_topics(corpus[i])]) for i in range(len(corpus))]

# Print the length of doc_topics
### YOUR CODE HERE (1 line)

df['topic_num'] = doc_topics

Before we look at documents, let's see if some topics captured documents with more positive ratings while others captured documents with more negative ratings. We will do this by calculating the average rating of all the documents in each topic.

Before running the cell below, look back up at the topics. Which topics seem like they are more positive? More negative?

Then run the cell below and look at the "avg_star_rating". Is it low or high for the topics you expected? (Note: "avg_star_rating" might be fairly high (>3) for all the topics because Amazon tends to have a lot more positive reviews than negative reviews)

In [ ]:
# What is the average rating in each topic? The number of documents in that topic?
topic_results = pd.DataFrame(df.groupby('topic_num').mean()['star_rating']).join(pd.DataFrame(df.groupby('topic_num').count()['review_body']))
topic_results =  topic_results.rename(columns={'review_body':'number_of_texts', 'star_rating':'avg_star_rating'})
topic_results

Now let's look at some texts for a particular topic. Do the texts seem to have to do with what you thought they did, based on the words in the topic?

Play around with the topic number in the cell below, and see whether the texts seem to fit with the topic.

In [ ]:
topic_num=3
rands = [random.randint(0, len(df[df['topic_num']==topic_num])) for i in range(5)]
for i in rands:
    print(df[df['topic_num']==topic_num].iloc[i]['review_body'])
    print()

As you can tell based on our topics above, one drawback of LDA is that the topics are not always very interpretable. This a a definite problem for short texts like our Amazon reviews, because LDA considers documents to be composed of multiple topics and tries to find the words that commonly co-occur. On our data in particular, you will likely find that the labels you came up with for the topics don't fit with most of the texts for which that topic has the highest probability.

While the results we saw on our data weren't great, LDA can actually be a very powerful tool for longer documents, or for a corpus that contains evident structure, such as documents in several different domains having domain-specific vocabulary. I recently obtained very good results using topic modelling on the paragraphs of data scientist job descriptions. The model distinguished quite well between the various types of paragraphs, such as "about the company", "skills", "equal opportunity".

### b) Most important phrases based on frequency

We will now try an approach I recently used in project work. We will start by finding the phrases or n-grams that occur most frequently. Then, we will see if "themes" emerge when we cluster these phrases. Finally, we can calculate how frequently each theme occurs in high-rating reviews and low-rating reviews to get a good view of what people like and don't like about the product.

#### What is CountVectorizer?

Sklearn's CountVectorizer creates a vector consisting in the counts of each word in the vocabulary for each text. For example, if our corpus was ["The brown dog jumps", "I like the brown house", "A brown dog runs to the brown house"], the vocabulary (after removing stopwords) is ["brown", "dog", "jumps", "like", "house", "runs"], and the vectors would be:

[[1, 1, 1, 0, 0, 0],

 [1, 0, 0, 1, 1, 0], 
 
 [2, 1, 0, 0, 1, 1]].

Our `find_top_phrases()` function uses `CountVectorizer()` to find the two to six word phrases that occur most frequently in our corpus.

### Find Common Phrases
* Overall
* For each rating

In [ ]:
def subset_data(df, col, value):
    """Subsets the data based on a certain value of a certain column."""
    df = df.copy()
    df = df[df[col]==value]

    return df

def find_top_phrases(data, text_col, rating_col='star_rating', rating_value=5, min_df=5):
        """Returns list of most frequent phrases in a subset of a dataframe, as well as metadata on the subset.
        
        data: Dataframe containing a text column
        text_col: Name of dataframe's text column
        rating_col: Name of the column containint the ratings (or some other categorical variable to subset on)
        rating_value: Value of the rating column on which to subset the data
        min_df: (int) Minimum number of texts that phrases must occur in
        """
        from sklearn.feature_extraction.text import CountVectorizer
        
        if rating_value:
            if rating_value not in data[rating_col].unique():
                rating_value = input(f"Which value of the target do you want to analyze? {data[rating_col].unique()}")

            print(f"Subsetting data on {rating_col}=={rating_value}")
            data = data[data[rating_col]==rating_value].copy()
        print("Number of texts:", len(data))
        if len(data)==0:
            return f"No texts with rating={rating_value} in the subset. Try again with a different rating_value"
        
        # Use cleaned text or raw text
        if text_col+"_cleaned" in data.columns:
            cleaned_text_col = text_col+"_cleaned"
        else:
            print("Using original (raw) text_col")
            cleaned_text_col = text_col
                                 
        # Find top phrases
        cv22 = CountVectorizer(min_df=min_df, ngram_range=(2,2), max_features=100)
        cv22.fit_transform(data[cleaned_text_col].to_list())
        phrases22 = cv22.get_feature_names()

        cv36 = CountVectorizer(min_df=min_df, ngram_range=(3,6), max_features=200)
        cv36.fit_transform(data[cleaned_text_col].to_list())
        phrases36 = cv36.get_feature_names()

        phrases_all = phrases22 + phrases36
        print("Number of phrases:", len(phrases_all))

        metadata = f"""Data analyzed: Subset of data where {rating_col}={rating_value}. 
                    All phrases are in at least {min_df} documents. Number of phrases: {len(phrases_all)}"""
        print(metadata)
        
        return phrases_all, metadata

In [ ]:
# Call find_top_phrases() on each of the rating values to extract phrases from
# reviews with each rating:
phrase_dct = {}
for r in sorted(df['star_rating'].unique()):
    phrases, meta = find_top_phrases(df, 'review_body', rating_value=r, min_df=4)
    phrase_dct[r] = phrases

In [ ]:
print("Number of phrases extracted from reviews with rating=1:", len(phrase_dct[1]))
print("First 5 phrases (alphabetically), extracted from reviews with rating=1:", phrase_dct[1][:5])

### Clustering Phrases into "Themes"

Now that we have the most frequent phrases for each rating, we can cluster them and see what themes emerge. Remember, the goal is to gain a high-level understanding from the texts, while reading through as little of the text as possible.

We will use K-Means clustering on the phrases. All machine learning algorithms take a numerical vector as input. Therefore, we have to vectorize our phrases. We will again use `CountVectorizer()`. With `CountVectorizer()`, each phrase will have its own numerical vector consisting in the number of times each word in the vocabulary appears in that phrase.

K-Means will cluster on these vectors, where phrases that contain some of the same words will be more similar than phrases that contain completely different words and so will likely be in the same cluster.

Let's take a look at what the `CountVectorizer()` vectors look like:

In [ ]:
cv = CountVectorizer()
vect = cv.fit_transform(phrase_dct[1])
pd.DataFrame.sparse.from_spmatrix(vect, columns = sorted(list(cv.vocabulary_.keys())), index=phrase_dct[1]).head()#.to_csv("Phrase_CV_Matrix.csv")

Each column is a word in the vocabulary, each row is a phrase, and the values are the number of times the word occurs in the phrase.

The `cluster_phrases()` function below vectorizes our phrase list and clusters the phrases into the number of clusters specified in the "n_clusters" parameter. Each cluster receives a title, which is merely the word showing up most frequently in that cluster.

In [ ]:
def _get_cluster_names(phrases_clustered):
    names = []
    for cluster in phrases_clustered:
        all_words = pd.Series(' '.join(phrase for phrase in cluster).split())
        top_word = all_words.value_counts().index[0]
        names.append(top_word.title())

    return names

def cluster_phrases(phrase_list, n_clusters=10):
    """K-Means clustering of top phrases. Returns nested list of phrases.

    phrase_list: list of phrases
    n_clusters: (int) number of clusters
    """

    from sklearn.cluster import KMeans
    cv = CountVectorizer()
    vect = cv.fit_transform(phrase_list)
    km = KMeans(n_clusters=n_clusters, random_state=12)
    clusters = km.fit_predict(vect)

    phrase_df = pd.DataFrame({"phrase":phrase_list, "cluster":clusters}, index=range(len(phrase_list)))
    phrases_clustered = []
    for c in sorted(phrase_df['cluster'].unique()):
        cluster_phrases = phrase_df[phrase_df['cluster']==c]['phrase']
        phrases_clustered.append(cluster_phrases.to_list())

    names = _get_cluster_names(phrases_clustered)

    return phrases_clustered, names

Let's take a look at our cluster names where the rating is 1 and where the rating is 5:

In [ ]:
# Clusters for phrases where the rating is 1
clusters1, names1 = cluster_phrases(phrase_dct[1], n_clusters=10)
print("Rating=1:", names1)
clusters5, names5 = cluster_phrases(phrase_dct[5], n_clusters=10)
print("Rating=5", names5)

While the names themselves say quite a bit, if we look at the phrases in a cluster, we can gain a greater understanding of the themes that have emerged.

In [ ]:
print(names1[6])
print(clusters1[6])

Let's also cluster all the phrases we discovered:

In [ ]:
# Cluster on all the phrases (from all the reviews)
phrases_all = list(set([y for x in phrase_dct.values() for y in x]))
clusters_all, names_all = cluster_phrases(phrases_all, n_clusters=30)

print("Cluster names from clustering all phrases:", names_all)

### Hit Rate Analysis

Now we will visualize how often phrases from these clusters occur in reviews with extreme ratings (rating=1 or rating=5). We call the percent of documents that contain any phrase from the cluster the "hit rate". One limitation of this approach is that it doesn't contain information on which phrases are occurring frequently. It might happen that positive documents have a high hit rate on a cluster extracted from negative documents because of a few particular "positive" phrases that happened to come up a few times in the negative reviews.

In [ ]:
def _hit_rate(phrase_lst, text_lst):
    hits = 0
    for t in text_lst:
        for p in phrase_lst:
            if p in t:
                hits += 1
                break

    hit_rate = hits/len(text_lst)

    return hit_rate

def compare_hit_rates(df, text_col, clustered_phrase_lst, cluster_names=None, subset_col='star_rating', subset_values=[1,5]):
    """Returns hit rate for each phrase cluster for specified values of the subset_col.

    clustered_phrase_lst: Nested list of phrases
    cluster_names: List of cluster names. If None, names will be the most common word in each cluster.
    subset_col: column for subsetting data (usually target_col). If None, will be target_col
    subset_values: (list of ints) Values of the subset column on which to calculate hit rates.
    df: (dataframe, optional). If None, will be self.dataframe
    """

    cleaned_text_col = text_col + '_cleaned'

    hit_rates_low = {}
    hit_rates_high = {}

    if type(cluster_names)==type(None):
        cluster_names = range(len(clustered_phrase_lst))

    assert len(cluster_names) == len(clustered_phrase_lst)
    if len(subset_values)>2:
        val_low = min(subset_values)
        val_high=max(subset_values)
        print("Using minimum and maximum values:", val_low, val_high)
    else:
        val_low = subset_values[0]
        val_high = subset_values[1]

    data_low = df[df[subset_col]==val_low][cleaned_text_col]
    data_high = df[df[subset_col]==val_high][cleaned_text_col]

    for i in range(len(clustered_phrase_lst)):
        hit_rates_high[cluster_names[i]] = _hit_rate(clustered_phrase_lst[i], data_high)
        hit_rates_low[cluster_names[i]] = _hit_rate(clustered_phrase_lst[i], data_low)

    return hit_rates_low, hit_rates_high

def plot_hit_rates(hit_rates_low, hit_rates_high):
    """Grouped bar plot of hit rates for each cluster for 'low' and 'high' subsets

    hit_rates_low: dictionary of {cluster_name: hit rate} for 'low' subset'
    hit_rates_high: dictionary of {cluster_name: hit rate} for 'high' subset

    """

    # Based on https://www.geeksforgeeks.org/create-a-grouped-bar-plot-in-matplotlib/
    low=1
    high=5

    # create data
    x = np.arange(len(hit_rates_low.values()))
    y_low = list(hit_rates_low.values())
    y_high = list(hit_rates_high.values())
    width = 0.4

    # plot data in grouped manner of bar type
    plt.figure(figsize=(12,6))
    plt.bar(x-width, y_low, width, color='red', label=f'{low}', align='edge')
    plt.bar(x, y_high, width, color='green', label=f'{high}', align='edge')
    plt.ylim(0,1)

    plt.xticks(x, list(hit_rates_low.keys()), rotation=90)

    plt.xlabel("Cluster")
    plt.ylabel("Cluster Hit Rate")
    plt.title("Proportion of Documents Containing One or More Phrases from the Cluster")

    plt.legend()
    plt.show()

First, let's take a look at the clusters extracted from the dataset as a whole. Which themes have a high red bar but low green bar (i.e. are mainly associated with negative reviews)? Which themes have a high green bar but low red bar (i.e. are mainly associated with positive reviews? Which themes occur at similar rates in positive and in negative reviews?

Feel free to dig deeper and look at the phrases contained in a few clusters.

##### Hit Rates All Phrases (Clustered)

In [ ]:
hit_rates_low, hit_rates_high = compare_hit_rates(df, 'review_body', clusters_all, names_all)

In [ ]:
plot_hit_rates(hit_rates_low, hit_rates_high)

In [ ]:
# Choose one of the cluster names
name = "Job" ### YOUR CODE HERE - replace "Job" with one of the names in names_all
try:
    assert name in names_all
except:
    name = input(f"Please choose one of the following: {names_all}\n")

# Find the index of the variable "name" in the list names_all
idx = None ### YOUR CODE HERE (replace None)

# Print the phrases in the cluster corresponding to the variable "name"
### YOUR CODE HERE (1 line)

##### Phrases from Star_Rating = 1

In [ ]:
# Show the hit rates for phrases extracted from extreme negative reviews (rating=1). Hint: use clusters1 and names1
### YOUR CODE HERE (2 lines)

##### Phrases from Star_Rating = 5

In [ ]:
# Show the hit rates for phrases extracted from extreme positive reviews (rating=1).
### YOUR CODE HERE (2 lines)

### Average Rating for Each Phrase or Theme

In [ ]:
def find_hits(df, text_col, target_col, phrase):
    dct = {}
    for value in sorted(df[target_col].unique()):
        dct[value] = 0
        texts = df[df[target_col]==value][text_col]
        for t in texts:
            if phrase in t:
                dct[value] += 1
    return dct

def avg_rating(phrase_rating_hits_dct):
    total_hits = np.sum(list(phrase_rating_hits_dct.values()))
    return np.sum([k*v for k, v in phrase_rating_hits_dct.items()])/total_hits

print(avg_rating(find_hits(df, 'review_body_cleaned', 'star_rating', 'works well')))
print(avg_rating(find_hits(df, 'review_body_cleaned', 'star_rating', 'take back')))
print(avg_rating(find_hits(df, 'review_body_cleaned', 'star_rating', 'cheap plastic')))

Calculate average rating for each theme, as we did for the topics from the topic model:

In [ ]:
cluster_hits_dcts = {}
for cluster, name in zip(clusters_all, names_all):
    dct = {1:0, 2:0, 3:0, 4:0, 5:0}
    for phrase in cluster:
        dct_phrase_hits = find_hits(df, 'review_body_cleaned', 'star_rating', phrase)
        for k, v in dct_phrase_hits.items():
            dct[k] += v
    cluster_hits_dcts[name] = dct
    
cluster_hits_dcts
print("Average Ratings:")
avg_ratings_dct = {}
for k, v in cluster_hits_dcts.items():
    avg_ratings_dct[k] = round(avg_rating(v),3)
pd.DataFrame(data={"Theme":list(avg_ratings_dct.keys()), "Avg Rating":list(avg_ratings_dct.values())}).sort_values(by='Avg Rating')

#### Another Method for Discovering Themes Associated with Positive and Negative Reviews

Instead of merely using the phrases extracted from reviews with a specific rating as we did above, here we consider a phrase "positive" or "negative" based on the average rating of all the reviews in which it occurs. We then cluster and plot the hit rates of these positive and negative phrases similarly to before.

In [ ]:
ratings = []
percents_all = []

# Make list of all phrases extracted from reviews of each rating
phrase_lst = list(set([y for x in phrase_dct.values() for y in x]))

for p in phrase_lst:
    # Create dictionary of number of times each phrase occurs in reviews of each rating
    dct = find_hits(df, 'review_body_cleaned', 'star_rating', p)
    
    # Calculate distribution of phrase occurrences across the ratings (Percent of total occurrences by rating)
    total_texts_containing_phrase = np.sum(list(dct.values()))
    percents = {}
    for k, v in dct.items():
        percents[k] = v/total_texts_containing_phrase
    ratings.append(avg_rating(dct))
    percents_all.append(percents)
    
rating_dist_df = pd.DataFrame({'phrase':phrase_lst, 'rating':ratings, 'distribution':percents_all}, 
                   index=range(len(phrase_lst))).sort_values(by='rating')
rating_dist_df.head()

In [ ]:
phrases_good = rating_dist_df[rating_dist_df['rating']>=4]['phrase'].to_list()
phrases_bad = rating_dist_df[rating_dist_df['rating']<=3]['phrase'].to_list()
len(phrases_good), len(phrases_bad)

In [ ]:
clusters_bad, names_bad = cluster_phrases(phrases_bad)
clusters_good, names_good = cluster_phrases(phrases_good)

print("Themes in negative reviews:", names_bad)
print("Themes in positive reviews:", names_good)

In [ ]:
print("Negative Themes")
hit_rates_low, hit_rates_high = compare_hit_rates(df, 'review_body', clusters_bad, names_bad)
plot_hit_rates(hit_rates_low, hit_rates_high)

In [ ]:
print("Positive Themes")
hit_rates_low, hit_rates_high = compare_hit_rates(df, 'review_body', clusters_good, names_good)
plot_hit_rates(hit_rates_low, hit_rates_high)

# Optional Section) EDA of Structured Metadata
Now that we've cleaned our data, we can explore it in a more meaningful way. What are some basic things might we want to know about our data?
* Number of documents
* Types of metadata we have on the documents (e.g. date, rating given, etc.)
* Distribution of metadata values

### Number of Documents

In [ ]:
print("Number of Documents:", len(df))

### Metadata

In [ ]:
print(df.dtypes)

### Time EDA

In [ ]:
# Convert "review_date" to datetime and find the range of our data
df['review_date'] = pd.to_datetime(df['review_date'])
print(f"Data is from {df['review_date'].min().date()} to {df['review_date'].max().date()}")

In [ ]:
# How many reviews per year?
df['year'] = df['review_date'].map(lambda x: x.year)
plt.bar(df['year'].value_counts().index, df['year'].value_counts().values)
plt.title("Review Year")
plt.ylabel('Number of Reviews')
plt.show()

### Ratings EDA

In [ ]:
# Mean rating over time
plt.plot(df.groupby('year').mean()['star_rating'].index, df.groupby('year').mean()['star_rating'])
plt.ylim(1,5.5)
plt.title("Mean Ratings Over Time")
plt.ylabel('Rating')
plt.show()

In [ ]:
# Overall rating distribution
def plot_ratings_dist(df, rating_col, year=None):
    if year:
        df = df[df['year']==year].copy()
    plt.bar(df.groupby([rating_col]).count()['year'].index, df.groupby([rating_col]).count()['year'].values)
    plt.show()


plot_ratings_dist(df, 'star_rating')

In [ ]:
# Distribution of ratings by year
ncols = 3
fig, ax = plt.subplots(nrows=5, ncols=ncols, figsize=(15,15))
rating_col='star_rating'

i = 0
j = 0
for y in sorted(df['year'].unique()):
    df_year = df[df['year']==y]
    gpd = df_year.groupby([rating_col]).count()['year']
    
    idx_lst = []
    zero_lst = []
    for idx in set([1,2,3,4,5]) - set(gpd.index):
        idx_lst.append(idx)
        zero_lst.append(0)
    gpd = gpd.append(pd.Series(zero_lst, index=idx_lst)).sort_index()
    
    ax[i,j].bar(gpd.index, gpd.values)
    ax[i,j].set_title(y)
    if j == ncols-1:
        i += 1
        j = 0
    else:
        j += 1